In [1]:
!pip install trax
import numpy as np
import pandas as pd
from trax import layers as tl
from nltk.tokenize import word_tokenize
from trax.fastmath import numpy as np
import random as rnd
from trax.supervised.training import TrainTask,EvalTask,Loop
import trax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 637.9/637.9 kB 10.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.4.0 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
kfp 1.8.20 requires google-api-python-client<2,>=1.7.8, but you have google-api-python-client 2.86.0 which is incompatible.
kfp 1.8.20 requires PyYAML<6,>=5.3, but you have pyyaml 6.0 which is incompatible.
gcsfs 2023.3.0 requires fsspec==2023

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data=pd.read_csv(r'/kaggle/input/ner-dataset/ner_datasetreference.csv',encoding = "ISO-8859-1")

In [3]:
data.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [4]:
index=data.dropna().index
index

Int64Index([      0,      24,      54,      68,      83,     108,     132,
                153,     181,     196,
            ...
            1048375, 1048400, 1048422, 1048438, 1048472, 1048501, 1048521,
            1048545, 1048556, 1048567],
           dtype='int64', length=47959)

In [5]:
sentence=[]
pos=[]
for idx,i in enumerate(index):
    one_Sentence=''
    one_pos=''
    if i ==index[-1]:
        break
    for i in range(index[idx],index[idx+1]):
        one_Sentence+=' '+data.iloc[i]['Word']
        one_pos+=' '+data.iloc[i]['POS']
    sentence.append(one_Sentence)
    pos.append(one_pos)

In [6]:
print(sentence[6])
print(pos[6])
data.iloc[index[6]:index[7]]

 The London march came ahead of anti-war protests today in other cities , including Rome , Paris , and Madrid .
 DT NNP NN VBD RB IN JJ NNS NN IN JJ NNS , VBG NNP , NNP , CC NNP .


,Sentence #,Word,POS,Tag
132,Sentence: 7,The,DT,O
133,NaN,London,NNP,B-geo
134,NaN,march,NN,O
135,NaN,came,VBD,O
136,NaN,ahead,RB,O
137,NaN,of,IN,O
138,NaN,anti-war,JJ,O
139,NaN,protests,NNS,O
140,NaN,today,NN,O
141,NaN,in,IN,O


In [7]:
print('data size=',len(sentence))
data_size=len(sentence)
train_data=sentence[:int(data_size*.8)]
test_data=sentence[len(train_data):int(data_size*.9)]
val_data=sentence[int(data_size*.9):]
train_label=pos[:int(data_size*.8)]
test_label=pos[len(train_data):int(data_size*.9)]
val_label=pos[int(data_size*.9):]
print(len(train_data),len(test_data),len(val_data))
print(len(train_label),len(test_label),len(val_label))
print(train_data[0])
print(train_label[0])

data size= 47958
38366 4796 4796
38366 4796 4796
 Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
 NNS IN NNS VBP VBN IN NNP TO VB DT NN IN NNP CC VB DT NN IN JJ NNS IN DT NN .


In [8]:
def word_to_number (data):
    vocab={'<pad>':0,'<unKnow>':1}
    for i in data:
        words=word_tokenize(i)
        for word in words:
            if word not in list(vocab):
                vocab[word]=len(vocab)
    return vocab


In [9]:
vocab_word=word_to_number(train_data)

In [11]:
vocab_word

{'<pad>': 0,
 '<unKnow>': 1,
 'Thousands': 2,
 'of': 3,
 'demonstrators': 4,
 'have': 5,
 'marched': 6,
 'through': 7,
 'London': 8,
 'to': 9,
 'protest': 10,
 'the': 11,
 'war': 12,
 'in': 13,
 'Iraq': 14,
 'and': 15,
 'demand': 16,
 'withdrawal': 17,
 'British': 18,
 'troops': 19,
 'from': 20,
 'that': 21,
 'country': 22,
 '.': 23,
 'Families': 24,
 'soldiers': 25,
 'killed': 26,
 'conflict': 27,
 'joined': 28,
 'protesters': 29,
 'who': 30,
 'carried': 31,
 'banners': 32,
 'with': 33,
 'such': 34,
 'slogans': 35,
 'as': 36,
 '``': 37,
 'Bush': 38,
 'Number': 39,
 'One': 40,
 'Terrorist': 41,
 'Stop': 42,
 'Bombings': 43,
 'They': 44,
 'Houses': 45,
 'Parliament': 46,
 'a': 47,
 'rally': 48,
 'Hyde': 49,
 'Park': 50,
 'Police': 51,
 'put': 52,
 'number': 53,
 'marchers': 54,
 'at': 55,
 '10,000': 56,
 'while': 57,
 'organizers': 58,
 'claimed': 59,
 'it': 60,
 'was': 61,
 '1,00,000': 62,
 'The': 63,
 'comes': 64,
 'on': 65,
 'eve': 66,
 'annual': 67,
 'conference': 68,
 'Britain': 69

In [12]:
def pos_to_number (label):
    vocab_pos={"<pad>":0}
    number_to_pos={0:'<pad>'}
    for poss in label:
        poss=poss.split()
        for pos in poss:
            if pos not in list(vocab_pos):
                vocab_pos[pos]=len(vocab_pos)
                number_to_pos[len(number_to_pos)]=pos
    return vocab_pos,number_to_pos


In [13]:
vocab_pos,number_to_pos=pos_to_number(train_label)

In [14]:
number_to_pos

{0: '<pad>',
 1: 'NNS',
 2: 'IN',
 3: 'VBP',
 4: 'VBN',
 5: 'NNP',
 6: 'TO',
 7: 'VB',
 8: 'DT',
 9: 'NN',
 10: 'CC',
 11: 'JJ',
 12: '.',
 13: 'VBD',
 14: 'WP',
 15: '``',
 16: 'CD',
 17: 'PRP',
 18: 'VBZ',
 19: 'POS',
 20: 'VBG',
 21: 'RB',
 22: ',',
 23: 'WRB',
 24: 'PRP$',
 25: 'MD',
 26: 'WDT',
 27: 'JJR',
 28: ':',
 29: 'JJS',
 30: 'WP$',
 31: 'RP',
 32: 'PDT',
 33: 'NNPS',
 34: 'EX',
 35: 'RBS',
 36: 'LRB',
 37: 'RRB',
 38: '$',
 39: 'RBR',
 40: ';',
 41: 'UH',
 42: 'FW'}

In [15]:
def tokenize_sentence(sentence,vocab):
    tokenize_sentence=[]
    words=word_tokenize(sentence)
    for word in words:
        
        tokenize_sentence.append(vocab.get(word,1))
    return tokenize_sentence
        

In [16]:
def gnerator_data(data,label):
    data_size=len(data)
    index=[*range(data_size)]
    idx=0
    while True:
        if idx>=data_size:
            idx=0
        rnd.shuffle(index)
        sentence=tokenize_sentence(data[index[idx]],vocab_word)
        pos=tokenize_sentence(label[index[idx]],vocab_pos)
        idx+=1
        X=np.array(sentence)
        Y=np.array(pos)
        yield ((X,Y))

In [17]:
X,Y=next(gnerator_data(train_data,train_label))
X.shape,Y.shape

((19,), (19,))

In [18]:
boundaries =  [8,   16,  32, 64, 128, 256, 512]
batch_sizes = [256,128, 64, 32, 16,    8,   4,  2]


# Create the generators.
train_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  # As before: count inputs and targets to length.
)(gnerator_data(train_data,train_label))

val_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  # As before: count inputs and targets to length.
)(gnerator_data(val_data,val_label))


test_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  # As before: count inputs and targets to length.
)(gnerator_data(test_data,test_label))

train_batch_stream = trax.data.AddLossWeights(id_to_mask=vocab_word['<pad>'])(train_batch_stream)
val_batch_stream = trax.data.AddLossWeights(id_to_mask=vocab_word['<pad>'])(val_batch_stream)


In [19]:
X,Y,M=next(train_batch_stream)
X.shape,Y.shape,M.shape

((64, 32), (64, 32), (64, 32))

In [36]:
X[10]

array([ 533,  171, 3073, 3074,  570, 2569,  143,  144,  679,    3, 2614,
         70, 5285, 4851,  398, 2514,    9, 2353,  409,  225,  107,   65,
         11, 7855,  202,   70, 2624,   23,    0,    0,    0,    0],
      dtype=int32)

In [37]:
Y[10]

array([11,  5,  5,  5, 18,  5, 25,  7,  1,  2,  5, 19, 11, 11, 11,  9,  6,
        7,  5,  2,  1,  2,  8, 11,  9, 19,  9, 12,  0,  0,  0,  0],
      dtype=int32)

In [39]:
M[10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
      dtype=float32)

In [45]:
def Model(d_model,n_layer):
    model=tl.Serial([tl.Embedding(len(vocab_word),d_model),
                         [tl.LSTM(d_model) for _ in range(n_layer)],
                          tl.Dense(len(vocab_pos)),
                          tl.LogSoftmax()

    ])
    return model

In [46]:
def train_model(train_batch_stream,val_batch_stream,n_steps,model,output_dir='./model'):
    train_task=TrainTask(train_batch_stream,
                         loss_layer=tl.CrossEntropyLoss(),
                         optimizer=trax.optimizers.Adam(0.001),
                         n_steps_per_checkpoint=100)
    
    eval_task=EvalTask(val_batch_stream,
                        metrics=[tl.CrossEntropyLoss(),tl.Accuracy()],
                      )
    
    training_loop =Loop(
        model, # A model to train
        train_task, # A train task
        eval_tasks = [eval_task], # The evaluation task
        output_dir = output_dir # The output directory
    )

    # Train with train_steps
    training_loop.run(n_steps = n_steps)
### END CODE HERE ###
    return training_loop
    


In [47]:
model=Model(512,3)
n_steps=int((len(train_data)/64))*3
print(n_steps)

training=train_model(train_batch_stream,val_batch_stream,n_steps,model,output_dir='./model')

1797


/opt/conda/lib/python3.10/site-packages/jax/_src/xla_bridge.py:658: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trax/layers/base.py:851: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip.GzipFile(fileobj=f, compresslevel=compresslevel) as gzipf:



Step      1: Total number of trainable weights: 22606379
Step      1: Ran 1 train steps in 21.36 secs
Step      1: train CrossEntropyLoss |  3.96340513


/opt/conda/lib/python3.10/site-packages/trax/supervised/training.py:1249: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip_lib.GzipFile(fileobj=f, compresslevel=2) as gzipf:


Step      1: eval  CrossEntropyLoss |  3.10591412
Step      1: eval          Accuracy |  0.12744437

Step    100: Ran 99 train steps in 172.66 secs
Step    100: train CrossEntropyLoss |  2.85941195
Step    100: eval  CrossEntropyLoss |  2.75693536
Step    100: eval          Accuracy |  0.16842106

Step    200: Ran 100 train steps in 169.16 secs
Step    200: train CrossEntropyLoss |  2.81053090
Step    200: eval  CrossEntropyLoss |  2.85680008
Step    200: eval          Accuracy |  0.14859095

Step    300: Ran 100 train steps in 163.34 secs
Step    300: train CrossEntropyLoss |  2.76927614
Step    300: eval  CrossEntropyLoss |  2.50585604
Step    300: eval          Accuracy |  0.23605151

Step    400: Ran 100 train steps in 166.01 secs
Step    400: train CrossEntropyLoss |  1.67825699
Step    400: eval  CrossEntropyLoss |  0.99980336
Step    400: eval          Accuracy |  0.74071527

Step    500: Ran 100 train steps in 172.40 secs
Step    500: train CrossEntropyLoss |  0.71134973
Step  

In [48]:
boundaries =  [8,   16,  32, 64, 128, 256, 512]
batch_sizes = [len(test_data)]*7


test_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  # As before: count inputs and targets to length.
)(gnerator_data(test_data,test_label))

train_batch_stream = trax.data.AddLossWeights(id_to_mask=vocab_word['<pad>'])(test_batch_stream)


In [49]:
X,Y,M=next(train_batch_stream)
X.shape,Y.shape,M.shape

((4796, 32), (4796, 32), (4796, 32))

In [50]:
def compute_accuracy(model,X,Y,M):
    prediction=model.eval_model(X)
    
    numerator=np.argmax(prediction,axis=-1)
    
    comprasion=np.sum(numerator==Y)
    
    denominator=np.sum(M)
    
    accuracy=comprasion/denominator
    return accuracy


In [51]:
print(f'Accuracy of Model is : {compute_accuracy(training,X,Y,M)}')

Accuracy of Model is : 0.9574911594390869


In [52]:
def pred(sentence,tokenize_sentence=tokenize_sentence):
    output=''
    tokenize_sentence=tokenize_sentence(sentence,vocab_word)
    tokenize_sentence=np.array(tokenize_sentence)
    tokenize_sentence=np.expand_dims(tokenize_sentence,axis=0)
    prediction=training.eval_model(tokenize_sentence)
    prediction=np.argmax(prediction,axis=-1)[0]
    for i in prediction:
        pos=number_to_pos[int(i)]
        output+=pos+' '
    return output


In [54]:
print(f'display actual output {pos[0]}')
print(f'display predicted output {pred(sentence[0])}')

display actual output  NNS IN NNS VBP VBN IN NNP TO VB DT NN IN NNP CC VB DT NN IN JJ NNS IN DT NN .
display predicted output NNS IN NNS VBP VBN IN NNP TO VB DT NN IN NNP CC NN DT NN IN JJ NNS IN DT NN . 
